# Chain Matrix Multiplication

## Problema
El problema de la Multiplicación de Cadenas de Matrices consiste en que, dado un conjunto de matrices con dimensiones específicas representadas en un arreglo, se busca determinar la secuencia óptima de multiplicación de estas matrices de manera que se minimice el número total de multiplicaciones de elementos requeridas. (GfG, 2022)

### Ejemplo del Problema
Buscamos multiplicar las siguientes matrices: 
$$
A_{(40x20)} \cdot B_{(20x30)} \cdot C_{(30x10)} \cdot D_{(10x30)}
$$

Existen varias formas de multiplicar estas matrices (por asociatividad):
$$
ABCD
$$
$$
(AB)CD
$$
$$
A(BC)D
$$
$$
(AB)(CD)
$$

Entonces, ¿cómo sabemos cuál es la mejor manera de multiplicar las matrices? ¿Cuál es la que minimiza el número de multiplicaciones de elementos que se necesita? Esto, con el objetivo de obtener el resultado de la multiplicación de una manera más rápida.

### Ejemplo de Solución
La resolución de este problema tiene un acercamiento con [programación dinámica](#programación-dinámica) y otro con el paradigma de [Divide y Vencerás](#divide-y-vencerás). Ambos funcionan de la misma manera. Proveemos un arreglo con las dimensiones de las matrices y se busca la secuencia óptima de multiplicación. Para el [ejemplo de problema](#ejemplo-del-problema) anterior, el arreglo sería ```[40, 20, 30, 10, 30]```. Este arreglo representa la multiplicación de las matrices $A_{(40x20)} \cdot B_{(20x30)} \cdot C_{(30x10)} \cdot D_{(10x30)}$.

Es necesario notar que el número de columnas de la matriz $i$ es igual al número de filas de la matriz $i+1$.

## Divide y Vencerás

Para complementar las secciones de aplicabilidad, es necesario comprender la idea en la que se estructura el problema.

Asumimos que existe un método $costo\_min(M_1, M_2)$, el cual retorna el costo mínimo de multiplicar la matriz $M_1$ con la matriz $M_2$.

Entonces, para una cadena de productos de matrices como $M_1 \cdot M_2 \cdot M_3 \cdot \ldots \cdot M_{n-1} \cdot M_n$ podemos obtener el costo mínimo de multiplicación de la cadena completa de matrices **dividiendo** el problema así:
$$
\begin{equation}
  \text{costo mínimo} = min
    \begin{cases}
        costo\_min(M_1, M_2 \cdot M_3 \ldots \cdot M_n) \\
        costo\_min(M_1 \cdot M_2 \ldots M_{n-1}, M_n)
    \end{cases}
\end{equation}
$$

Esto genera dos subproblemas, los cuales pueden ser descompuestos en otros subproblemas más pequeños bajo la misma idea hasta llegar a realizar la multiplicación de solamente dos matrices (caso base). En este punto, sabemos cuál es la cantidad mínima de multiplicaciones (**conquistamos**). Luego, en caso de ser necesario, **combinamos** con otro cálculo de $costo\_min$.  

### Algoritmo
    Función Principal: Recibe matrix_array que es una lista y devuelve un entero:
        Establecer array_len como la longitud de matrix_array
    
        Inicializar una tabla 2D de tamaño array_len x array_len, llena de infinito
    
        Iniciar el proceso recursivo desde la primera matriz hasta la última con la Función Recursiva pasando el arreglo con las dimensiones de las matrices, la tabla y los índices de inicio (1) y fin (array_len - 1)
    
    Fin de la Función
    
    Función Recursiva recibe matrix_array que es una lista, table que es una lista, i que es un entero, j que es un entero y devuelve un entero:
        Si i es igual a j, devolver 0
    
        Si la tabla[i][j] no es infinito, devolver tabla[i][j]
    
        Para k desde i hasta j:
            Establecer count como la suma de una llamada recursiva con (matrix_array, table, i, k), otra llamada recursiva con (matrix_array, tabla, k + 1, j) y el producto de matrix_array[i - 1], matrix_array[k] y matrix_array[j]
    
            Si esa suma (count) es menor que table[i][j], actualizar la tabla[i][j] a count
    
        Devolver table[i][j]
    
    Fin de la Función

### Aplicabilidad
El acercamiento utilizando DaC es aplicable para este problema, ya que existe la posibilidad de dividir el problema en subproblemas más pequeños que pueden resolverse de manera independiente. Siendo el enfoque justificable por la capacidad de dividir la cadena de matrices en partes más pequeñas, resolver cada parte de manera recursiva y luego combinar las soluciones de manera óptima para obtener el resultado final. Dicha aplicación conlleva dividir la cadena en dos subcadenas en cada paso recursivo, calcular el costo mínimo de multiplicación para cada subcadena y combinar las soluciones para obtener el costo total mínimo (GfG, 2022).

### Implementación

In [80]:
def _MatrixChainOrderRecursive(matrix_array: list, table: list, i: int, j: int) -> int:
    """
    Recursive helper function to find the minimum number of multiplications needed to multiply the chain of matrices.
    :param matrix_array: The dimensions of the matrices in the chain.
    :param table: The DP table to store the minimum cost of matrix multiplication.
    :param i: The starting index of the chain.
    :param j: The ending index of the chain.
    :return: The minimum number of multiplications needed.
    """
    # base case: when the chain consists of a single matrix, no multiplication is needed
    if i == j:
        return 0

    # if the solution for this subproblem has already been calculated, return it
    # where the results of expensive function calls and reuse the results are stored (memoization)
    if table[i][j] != float('inf'):
        return table[i][j]

    # --- DIVIDE ---
    # recursively find the minimum multiplication cost by dividing the problem into smaller subproblems and trying all possible places to split the chain
    for k in range(i, j):
        count = _MatrixChainOrderRecursive(matrix_array, table, i, k)  # cost of multiplying matrices from i to k
        count += _MatrixChainOrderRecursive(matrix_array, table, k + 1, j)  # cost of multiplying matrices from k+1 to j
        count += matrix_array[i - 1] * matrix_array[k] * matrix_array[
            j]  # cost of multiplying the two resulting matrices

        # --- COMBINE AND CONQUER ---
        # update dp[i][j] with the minimum cost found so far
        if count < table[i][j]:
            table[i][j] = count

    # return the minimum cost for multiplying matrices from i to j
    return table[i][j]

In [81]:
def MatrixChainOrderDnC(matrix_array: list) -> int:
    """
    Returns the minimum number of multiplications needed to multiply the chain of matrices. Uses Divide and Conquer
    :param matrix_array: The dimensions of the matrices in the chain.
    :return: The minimum number of multiplications needed.
    """
    array_len = len(matrix_array)

    # initialize the DP table with inf, indicating that the minimum cost has not been calculated yet
    table = [
        [float('inf') for _ in range(array_len)]
        for _ in range(array_len)
    ]

    # start the recursive process from the first matrix to the last
    return _MatrixChainOrderRecursive(matrix_array, table, 1, array_len - 1)

## Programación Dinámica
El acercamiento de programación dinámica permite que cada vez que se llega a realizar al caso base, es almacenado en una tabla para evitar los cálculos redundantes.

Para clarificar de manera visual, observar el siguiente árbol de recursividad. En él se pueden apreciar dos aspectos claves para poder aplicar programación dinámica, que son:
- *Subestructuras óptimas*: Descomponemos los problemas grandes en subproblemas más pequeños, los cuales se resuelven para encontrar el número mínimo de multiplicaciones. Podemos decir que el problema tiene subestructuras óptimas.
- *Traslapo de subproblemas*: En el árbol de recursión se aprecia como hay ciertos subproblemas que se repiten, ocupando recursos computacionales adicionales.

![Árbol de Recursión para el Acercamiento dinámico](media/recursion_tree.jpeg)

La manera en la que se ha resuelto es a través de un acercamiento _top-down_, pues primero se verifica si ya ha sido resuelto para devolver un valor. Si no ha sido resuelto, se hace el cálculo y se almacena la respuesta para referencia futura.

### Algoritmo
    Recibe matrix_array que es una lista con las dimensiones de las matrices a multiplicar

        Establecer array_len como la longitud de matrix_array

        Inicializar una tabla 2D de tamaño (array_len x array_len), llena de 0s

        Para current_length desde 2 hasta array_len:
            Para i desde 1 hasta array_len - current_length + 1:
                Establecer j como i + current_length - 1
                Establecer la posición tabla[i][j] como infinito

                Para current_point desde i hasta j:
                    Calcular q como la suma de table[i][current_point], table[current_point + 1][j] y el producto de matrix_array[i - 1], matrix_array[current_point] y matrix_array[j]

                    Si q es menor que table[i][j], actualizar table[i][j] a q

        Devolver table[1][array_len - 1], pues ahí se encuentra el costo mínimo de la cadena completa de matrices
        
    Fin

### Aplicabilidad
La utilización de programación dinámica es aplicable para este problema, debido a la presencia de subestructura óptima y subproblemas superpuestos en el problema. Este enfoque se justifica teóricamente por la capacidad de descomponer el problema en subproblemas más pequeños, calcular y almacenar las soluciones de estos de manera eficiente y luego la combinación de estas para obtener la solución óptima global.  Dicha aplicación conlleva construir una tabla de programación dinámica que almacene las soluciones óptimas para subproblemas específicos, calculando iterativamente las soluciones basadas en las soluciones de subproblemas más pequeños y utilizando la memoización para evitar cálculos adicionales innecesarios (More, 2021).

### Implementación

In [79]:
def MatrixChainOrderDP(matrix_array: list) -> int:
    """
    Returns the minimum number of multiplications needed to multiply the chain of matrices. Uses Dynamic Programming
    :param matrix_array: The dimensions of the matrices in the chain.
    :return: The minimum number of multiplications needed.
    """
    array_len = len(matrix_array)

    # Initialize a 2D table to store the minimum cost of matrix multiplication.
    table = [
        [0 for _ in range(array_len)]
        for _ in range(array_len)
    ]

    # Current_length represents the chain length being considered, the algorithm gradually increases the chain length from 2 to n
    for current_length in range(2, array_len):
        for i in range(1, array_len - current_length + 1):
            # calculate the ending matrix index in the chain based on current_length and i
            j = i + current_length - 1
            # initialize the cost to infinity before finding the minimum.
            table[i][j] = float('inf')

            # chain split at every possible point, calculating he cost for each one, and keep track of the minimum cost found
            for current_point in range(i, j):
                # calculate cost of splitting at k, including the cost of multiplying the two resulting chains
                q = table[i][current_point] + table[current_point + 1][j] + matrix_array[i - 1] * matrix_array[
                    current_point] * matrix_array[j]

                # update dp[i][j] if a lower cost is found
                if q < table[i][j]:
                    table[i][j] = q

    # dp[1][n-1] holds the minimum cost for the full chain of matrices
    return table[1][array_len - 1]

## Pruebas

In [82]:
# dimensions of matrices in the chain
# matrix_array = [10, 20, 30]  # A(10x20), B(20x30) → 6_000
# matrix_array = [40, 20, 30, 10, 30]  # A(40x20), B(20x30), C(30x10), D(10x30) → 26_000
matrix_dimensions = [10, 20, 30, 40, 30]  # A(10x20), B(20x30), C(30x40), D(40x30) → 30_000

print(f"Divide & Conquer:\t\tMinimum number of multiplications is: {MatrixChainOrderDnC(matrix_dimensions)}")
print(f"Dynamic Programming:\tMinimum number of multiplications is: {MatrixChainOrderDP(matrix_dimensions)}")

Dynamic Programming:	Minimum number of multiplications is: 30000
Divide & Conquer:		Minimum number of multiplications is: 30000


## Referencias
- GeeksforGeeks [GfG]. (2022). _Matrix Chain Multiplication | DP-8. GeeksforGeeks_. [https://www.geeksforgeeks.org/matrix-chain-multiplication-dp-8/](https://www.geeksforgeeks.org/matrix-chain-multiplication-dp-8/)
- More, C. (2019). Matrix Chain Multiplication using Dynamic Programming. Medium. [https://medium.com/@hichetanmore/matrix-chain-multiplication-using-dynamic-programming-22a137df955f](https://medium.com/@hichetanmore/matrix-chain-multiplication-using-dynamic-programming-22a137df955f)